# Homework Workshop 1

Importo los modulos necesarios

In [1]:
import dlt
import requests
import json

from dlt.destinations import qdrant

## Question 1. dlt Version

In [2]:
!dlt --version

dlt 1.12.3


## Question 2. dlt pipeline

In [3]:
@dlt.resource
def zoomcamp_data():
    docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            yield doc

Se recomienda leer sobre la [integración dlt + qdrant](https://dlthub.com/docs/dlt-ecosystem/destinations/qdrant)

Configura el destino en el archivo de secretos de dlt (por defecto en `~/.dlt/secrets.toml`). Agrega la siguiente sección:

```toml
[destination.qdrant]
qd_location = "https://your-qdrant-url"
[destination.qdrant.credentials]
api_key = "your-qdrant-api-key"
```

Si no se proporciona ninguna configuración, se usará por defecto http://localhost:6333 sin clave de API.

También puedes usar Qdrant sin backend, guardando la base de datos localmente en un archivo.

```toml
[destination.qdrant]
qd_path = "db.qdrant"
```

In [8]:
! tree .dlt

.dlt
└── secrets.toml

0 directories, 1 file


In [9]:
! cat .dlt/secrets.toml

[destination.qdrant]
qd_path = "db.qdrant"

In [4]:
qdrant_destination = qdrant()

In [5]:
pipeline = dlt.pipeline(
    pipeline_name="zoomcamp_pipeline",
    destination=qdrant_destination,
    dataset_name="zoomcamp_tagged_data"
)
load_info = pipeline.run(zoomcamp_data())
print(pipeline.last_trace)

Run started at 2025-07-06 11:45:22.793510+00:00 and COMPLETED in 3 minutes and 26.10 seconds with 4 steps.
Step extract COMPLETED in 0.81 seconds.

Load package 1751802325.993873 is EXTRACTED and NOT YET LOADED to the destination and contains no failed jobs

Step normalize COMPLETED in 0.13 seconds.
Normalized data for the following tables:
- zoomcamp_data: 948 row(s)
- _dlt_pipeline_state: 1 row(s)

Load package 1751802325.993873 is NORMALIZED and NOT YET LOADED to the destination and contains no failed jobs

Step load COMPLETED in 3 minutes and 21.96 seconds.
Pipeline zoomcamp_pipeline load step completed in 3 minutes and 21.94 seconds
1 load package(s) were loaded to destination qdrant and into dataset zoomcamp_tagged_data
The qdrant destination used /home/joviedo/github-repos/llm-zoomcamp-knowledge-base/hw_workshop1/notebook/db.qdrant location to store data
Load package 1751802325.993873 is LOADED and contains no failed jobs

Step run COMPLETED in 3 minutes and 26.10 seconds.
Pipel

## Question 3. Embeddings

In [13]:
! ls db.qdrant

collection  meta.json


In [14]:
! tree db.qdrant

db.qdrant
├── collection
│   ├── zoomcamp_tagged_data
│   │   └── storage.sqlite
│   ├── zoomcamp_tagged_data__dlt_loads
│   │   └── storage.sqlite
│   ├── zoomcamp_tagged_data__dlt_pipeline_state
│   │   └── storage.sqlite
│   ├── zoomcamp_tagged_data__dlt_version
│   │   └── storage.sqlite
│   └── zoomcamp_tagged_data_zoomcamp_data
│       └── storage.sqlite
└── meta.json

6 directories, 6 files


In [16]:
! cat db.qdrant/meta.json

{"collections": {"zoomcamp_tagged_data": {"vectors": {"fast-bge-small-en": {"size": 384, "distance": "Cosine", "hnsw_config": null, "quantization_config": null, "on_disk": null, "datatype": null, "multivector_config": null}}, "shard_number": null, "sharding_method": null, "replication_factor": null, "write_consistency_factor": null, "on_disk_payload": null, "hnsw_config": null, "wal_config": null, "optimizers_config": null, "init_from": null, "quantization_config": null, "sparse_vectors": null, "strict_mode_config": null}, "zoomcamp_tagged_data__dlt_loads": {"vectors": {"fast-bge-small-en": {"size": 384, "distance": "Cosine", "hnsw_config": null, "quantization_config": null, "on_disk": null, "datatype": null, "multivector_config": null}}, "shard_number": null, "sharding_method": null, "replication_factor": null, "write_consistency_factor": null, "on_disk_payload": null, "hnsw_config": null, "wal_config": null, "optimizers_config": null, "init_from": null, "quantization_config": null, "

In [37]:
file_path = 'db.qdrant/meta.json'
data = None

with open(file_path, 'r') as file:
    data = json.load(file)

In [46]:
for colection, value in data['collections'].items():
    print(f"Coleccion: {colection}, modelo de incrustación:{list(value['vectors'].keys())}")

Coleccion: zoomcamp_tagged_data, modelo de incrustación:['fast-bge-small-en']
Coleccion: zoomcamp_tagged_data__dlt_loads, modelo de incrustación:['fast-bge-small-en']
Coleccion: zoomcamp_tagged_data_zoomcamp_data, modelo de incrustación:['fast-bge-small-en']
Coleccion: zoomcamp_tagged_data__dlt_version, modelo de incrustación:['fast-bge-small-en']
Coleccion: zoomcamp_tagged_data__dlt_pipeline_state, modelo de incrustación:['fast-bge-small-en']
